In [17]:
import os
import pandas as pd
import xlrd
from pathlib import Path
import openpyxl
from openpyxl import load_workbook
import numpy as np
import re

In [18]:

    ##Getting the 1045 data
xlsx_file = Path('SimData', '/home/ygael/Projects/healthcare/data/2020_CDM_chargemaster/Alvarado Hospital/106370652_CDM_All_2020.xlsx')
wb_obj = openpyxl.load_workbook(xlsx_file) 

In [19]:
#Function to extract 1045 sheet name

def find1045(filepath):
    #read file
    xlsx_file = Path('SimData', filepath)
    wb_obj = openpyxl.load_workbook(xlsx_file)

    #get sheet names
    sheetnames=wb_obj.sheetnames

    #Create a keywords list to identify ABS 1045 forms
    keywords= ["common", "procedures", "1045", "form"]

    #Search for the sheet by keyword
    for name in sheetnames:
        wholename=name.lower()
        wordsInName=wholename.split()
        match =  any(item in wordsInName for item in keywords)
        if match==True:
            sheetname=(wb_obj[name])
            return(sheetname)
            break


In [20]:
#Function to extract all dataframes from a single xlsx file
def getdata(file):
    dflist=[]
    coordlist=[]
    sheet=find1045(file)
    alphabet=["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z"]
    found=False
    keywords1 = ["evaluation", "management", "services"]
    keywords2 = ["laboratory", "pathology", "services"]
    keywords3= ["radiology", "services"]
    keywords4 = ["medicine", "services"]
    keywords5 = ["surgery", "services"]
    keywords6 = ["other", "common", "outpatient", "procedures"]
    keywords7 = ["count", "of", "reported", "procedures",]
    keywordslists=[keywords1,keywords2,keywords3,keywords4,keywords5,keywords6,keywords7]
    for wordlist in keywordslists:
        for row in sheet.iter_rows():
                for cell in row:
                    heading = str(cell.value).lower()
                    wordsInHeading=heading.split()
                    match =  all(item in wordsInHeading for item in wordlist)
                    if match==True:
                        coord1=cell.coordinate
                        coordlist.append(coord1)
                        break
    for coordinate in coordlist[0:-1]:
        splitcoord=re.split('(\d+)', coordinate)
        nextbreak=coordlist[int(coordlist.index(coordinate)+1)]
        splitbreak=re.split('(\d+)', nextbreak)
        lastcell=f"{int(splitbreak[1])-1}"
        coord2=f"C{lastcell}"

        cellgroup=[coordinate,coord2]
        df=pd.DataFrame(sheet[f"{cellgroup[0]}:{cellgroup[1]}"])



        dflist.append(df)

    return(dflist)

  
    

In [21]:
#Function to convert cell object values to plaintext values
def convertvalue(x):
    return x.value

In [22]:
#Convert all dataframes from a single sheet into plaintext
sheet=find1045(xlsx_file)
masterlist=[]
rawlist=getdata(xlsx_file)
for i in range(len(rawlist)):
    df=rawlist[i].applymap(convertvalue)
            #add the hospital name as a variable
    masterlist.append(df)

In [23]:
#Get the column names from the dataframe
for df in masterlist:
    if len(df)>0:
        df.columns = df.iloc[0]
        df["Service Type"]=df.iloc[0,0]
        df.rename(columns={ df.columns[0]: "Service" }, inplace = True)
        df=df.drop(df.index[0], inplace=True)




In [24]:
#add the hospital name as a variable in every df in the converted list
for i in range(len(masterlist)):
        raw_name=sheet["A1"].value
        hos_name=raw_name.replace('Hospital Name: ', '')
        masterlist[i]["Hospital"]=hos_name


In [25]:
masterlist[0]

,Service,2019 CPT Code,Average Charge,Service Type,Hospital
1,"Emergency Room Visit, Level 2 (low to moderate...",99282,746.55,Evaluation & Management Services (CPT Codes 99...,Alvarado Hospital
2,"Emergency Room Visit, Level 3 (moderate severity)",99283,1051.05,Evaluation & Management Services (CPT Codes 99...,Alvarado Hospital
3,"Emergency Room Visit, Level 4 (high severity)",99284,1581.3,Evaluation & Management Services (CPT Codes 99...,Alvarado Hospital
4,"Outpatient Visit, established patient, 15 minutes",99213,None,Evaluation & Management Services (CPT Codes 99...,Alvarado Hospital


In [29]:
#get the names of all folders in the data directory
os.chdir("..")
os.chdir("data")
os.chdir("2020_CDM_chargemaster")



NameError: name 'chargemaster' is not defined

In [69]:
#get the names of all files from each directory, then test if each file has a 1045 in it.
dirlist = list(os. listdir())
for i in range(len(dirlist)):
    os.chdir(dirlist[i])
    for root, dirs, files in os.walk("."):
        for filename in files:
            print(filename)
    os.chdir("..")

106390923_CDM_All_2020.xlsx
106104023_CDM_All_2020.xls
106374465_Common25_2020.xlsx
106370730_CDM_2020.xlsx
106370730_PCT_CHG_2020.xlsx
106370730_Common25_2020.xlsx
Thumbs.db
106361339_CDM_All_2020.xlsx
Thumbs.db
106361339_Comments_2020.docx
106314024_PCT_CHG_2020.xls
106314024_PCT_CHG_2020-Cloud.xls
106314024_CDM_2020.xlsx
106314024_Common25_2020.xlsx
Thumbs.db
106560473_CDM_All_2020.xlsx
106190197_Common25_2020.xlsx
106190197_Comments_2020.docx
106190197_CDM_2020.xlsx
106331293_CDM_All_2020.xlsx
106015000_PCT_CHG_2020.xls
106015000_CDM_2020.xlsx
Thumbs.db
106015000_Common25_2020.xlsx
106391010_PCT_CHG_2020.xlsx
106391010_CDM_2020.xlsx
106391010_Comments_2020.doc
106391010_Common25_2020.xlsx
106420522_CDM_All_2020.xlsx
Thumbs.db
106560529_CDM_All_2020.xlsx
106560529_Comments_2020.docx
106370749_Comments_2020.docx
106370749_Common25_2020.xlsx
106370749_CDM_2020.xlsx
106301342_CDM_All_2020.xlsx
Thumbs.db
106560501_CDM_All_2020.xlsx
106190034_CDM_All_2020.xlsx
Thumbs.db
106450940_CDM_All

In [58]:
type(dirlist[i])

str

In [64]:
os.getcwd()

'/home/ygael/Projects/healthcare/data/2020_CDM_chargemaster'

In [63]:
os.chdir("..")